In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 6.5 MB 44.3 MB/s 
     |████████████████████████████████| 895 kB 64.7 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 55.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
import numpy as np
import pandas as pd
import time
import re
from random import sample

from sklearn.model_selection import train_test_split

In [ ]:
#Tempo de processamento
tempoInicial = time.time()

#Faz a leitura da base
df = pd.read_csv('dataset7DiasCompleto-En.v1.csv', sep=';')
df.describe()

colunaCorpus='titulo_processado'
colunaResultado='classe'

In [ ]:
randomState = sample(range(0, 1000), 1)[0];

#Definindo X, y
X = df[df.columns.difference([colunaResultado])]
y = df[colunaResultado]

#Separa base treinamento e teste
XTreino, XTeste, yTreino, yTeste = train_test_split(X, y, train_size=0.7, 
                                                    stratify=y, shuffle=True, 
                                                    random_state=randomState)
XTreino = XTreino[colunaCorpus].values
yTreino = yTreino.values
XTeste = XTeste[colunaCorpus].values
yTeste = yTeste.values

In [ ]:
import transformers

tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
corpus = XTreino
#corpus = XTreino[colunaCorpus]
maxlen = 512

In [ ]:
def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [],[],[]
    for s in range(len(sentences)):
        sentence = sentences[s]
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=512, pad_to_max_length=True, 
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32')

X = tokenize(corpus, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
import tensorflow as tf
from transformers import TFRobertaModel

## inputs
input_ids_in = tf.keras.layers.Input(shape=(512,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(512,), name='masked_token', dtype='int32') 

## pre-trained bert with config
transformer_model = TFRobertaModel.from_pretrained('roberta-base', output_hidden_states = False)
bert_out = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]

## fine-tuning
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(bert_out)
x = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.keras.layers.Dense(50, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(len(np.unique(yTreino)), activation='sigmoid')(x)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = x)

for layer in model.layers[:3]:
    layer.trainable = False

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])
model.summary()

Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 512)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 512)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_token[0][0]',            
 el)                            thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [ ]:
## encode y
#dic_y_mapping = {n:label for n,label in 
#                 enumerate(np.unique(yTreino))}
#inverse_dic = {v:k for k,v in dic_y_mapping.items()}
#yTreino = np.array([inverse_dic[y] for y in yTreino])

## train
training = model.fit(x=X, y=yTreino, batch_size=128, epochs=1, shuffle=True, 
                     verbose=1, validation_split=0.2)

38/38 [==============================] - 14171s 373s/step - loss: 1.0937 - accuracy: 0.5676 - val_loss: 0.8036 - val_accuracy: 0.7174


In [ ]:
corpus = XTeste
X = tokenize(corpus, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
## test
predicted_prob = model.predict(X)
predicted = [np.argmax(pred) for pred in predicted_prob]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(yTeste, predicted, labels=[0, 1, 2, 3, 4]))

              precision    recall  f1-score   support

           0       0.81      0.62      0.70       452
           1       0.58      0.80      0.68       629
           2       0.65      0.68      0.66       645
           3       0.79      0.67      0.72       600
           4       0.84      0.62      0.71       268

    accuracy                           0.69      2594
   macro avg       0.73      0.68      0.70      2594
weighted avg       0.71      0.69      0.69      2594



In [ ]:
print("\n--- %.2f minutos ---" % ((time.time() - tempoInicial) / 60))


--- 321.04 minutos ---
